In [1]:
# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning

In [ ]:
import torch
from torch import nn
import pytorch_lightning as pl
from torchmetrics.functional import accuracy
from torch.utils.data import random_split, DataLoader
from torchvision import transforms, datasets, models

In [3]:
pl.seed_everything(42)
max_epochs = 3
img_size = 224
val_pct = 0.2
batch_size = 64
lr = 3e-4

Global seed set to 42


In [4]:
class CIFAR10DataModule(pl.LightningDataModule):
    def __init__(self, data_dir, img_size, val_pct, batch_size):
        super().__init__()
        self.data_dir = data_dir
        self.T = transforms.Compose(
                    [
                    transforms.Resize((img_size, img_size)),
                    transforms.ToTensor()
                    ]
                )
        self.val_pct = val_pct
        self.batch_size = batch_size

    def prepare_data(self):
        datasets.CIFAR10(self.data_dir, train=True, download=True)
        datasets.CIFAR10(self.data_dir, train=True, download=True)

    def setup(self, stage=None):
        data = datasets.CIFAR10(self.data_dir, train=True, transform=self.T)
        val_len = int(val_pct * len(data))
        self.train_data, self.val_data = random_split(data, [len(data) - val_len, val_len])
        self.test_data = datasets.CIFAR10(self.data_dir, train=False, transform=self.T)

    def get_dataloader(self, data):
        return DataLoader(data, batch_size=self.batch_size, num_workers=2, pin_memory=True)

    def train_dataloader(self):
        return self.get_dataloader(self.train_data)

    def val_dataloader(self):
        return self.get_dataloader(self.val_data)

    def test_dataloader(self):
        return self.get_dataloader(self.test_data)

In [5]:
class Model(pl.LightningModule):
    def __init__(self, lr):
        super().__init__()
        self.resnet = models.resnet18(pretrained=False)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 10)

        self.lr = lr
        self.loss_fn = nn.CrossEntropyLoss()

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

    def forward(self, x):
        return self.resnet(x)

    def shared_step(self, batch, split):
        x, y = batch
        preds = self(x)
        loss = self.loss_fn(preds, y)
        acc = accuracy(preds, y)
        self.log(f'{split}_loss', loss, on_epoch=True, prog_bar=True)
        self.log(f'{split}_acc', acc, on_epoch=True, prog_bar=True)
        if split == 'train':
            return loss

    def training_step(self, batch, batch_idx):
        loss = self.shared_step(batch, 'train')
        return loss

    def validation_step(self, batch, batch_idx):
        self.shared_step(batch, 'val')

    def test_step(self, batch, batch_idx):
        self.shared_step(batch, 'test')

In [6]:
class PrintMetrics(pl.Callback):
    def on_train_epoch_end(self, trainer, pl_module):
        final_str = f"epoch: {trainer.current_epoch} | "
        for k, v in trainer.callback_metrics.items():
            if "train" in k and "epoch" in k:
                final_str += f"{k.replace('_epoch', '')}: {v.item():.4f} | "
            elif "val" in k:
                final_str += f"{k}: {v.item():.4f} | "
        print(final_str[:-3])

In [7]:
cifar10_dm = CIFAR10DataModule('data/', img_size, val_pct, batch_size)
model = Model(lr)

In [8]:
trainer = pl.Trainer(max_epochs=max_epochs, gpus=1, callbacks=[PrintMetrics()])
trainer.fit(model, cifar10_dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:168: LightningDeprecationWarning: DataModule property `has_prepared_data` was deprecated in v1.4 and will be removed in v1.6.
  'DataModule property `has_prepared_data` was deprecated in v1.4 and will be removed in v1.6.'


Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params
---------------------------------------------
0 | resnet  | ResNet           | 11.2 M
1 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Global seed set to 42


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:645: LightningDeprecationWarning: Relying on `self.log('val_loss', ...)` to set the ModelCheckpoint monitor is deprecated in v1.2 and will be removed in v1.4. Please, create your own `mc = ModelCheckpoint(monitor='your_monitor')` and use it as `Trainer(callbacks=[mc])`.
  "Relying on `self.log('val_loss', ...)` to set the ModelCheckpoint monitor is deprecated in v1.2"


epoch: 0 | val_loss: 1.3979 | val_acc: 0.5412 | train_loss: 1.2836 | train_acc: 0.5342


epoch: 1 | val_loss: 1.0421 | val_acc: 0.6477 | train_loss: 0.8176 | train_acc: 0.7118


epoch: 2 | val_loss: 0.9970 | val_acc: 0.6721 | train_loss: 0.5971 | train_acc: 0.7922



In [12]:
trainer.test(model, cifar10_dm)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:168: LightningDeprecationWarning: DataModule property `has_prepared_data` was deprecated in v1.4 and will be removed in v1.6.
  'DataModule property `has_prepared_data` was deprecated in v1.4 and will be removed in v1.6.'
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:438: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.6650999784469604, 'test_loss': 1.0200164318084717}
--------------------------------------------------------------------------------



/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:438: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  f"DataModule.{name} has already been called, so it will not be called again. "


[{'test_acc': 0.6650999784469604, 'test_loss': 1.0200164318084717}]